In [91]:
def get_canadian_companies():
    url = "https://en.wikipedia.org/wiki/List_of_companies_of_Canada"
    companies = pd.read_html(url)[1]  # Get just table 1
    # Clean up the column names (remove multi-index)
    companies.columns = companies.columns.get_level_values(0)
    # Keep only the columns we need
    companies = companies[['Name', 'Industry', 'Sector', 'Notes', 'Headquarters']].copy()
    return companies

# Get the data
companies_df = get_canadian_companies()
companies_df.head()

,Name,Industry,Sector,Notes,Headquarters
0,1-800-GOT-JUNK?,Industrials,Waste & disposal services,Junk removal,Vancouver
1,Norda Stelo,Industrials,Construction & Materials,Integrated projects,Quebec City
2,3Way International Logistics,Industrials,Business support services,Freight forwarding,Mississauga
3,A Buck or Two,Consumer services,Specialty retailers,Retail,Vaughan
4,A&W,Consumer services,Restaurants & bars,Fast food chain,North Vancouver


In [6]:
from pydantic import BaseModel
from typing import List
from openai import OpenAI
import os

class CompanyVariants(BaseModel):
    original_name: str
    positive_variants: List[str]
    negative_variants: List[str]


In [87]:
from pydantic import BaseModel
from typing import List
from openai import OpenAI
import os
import pandas as pd
from textdistance import jaro_winkler as jaro_winkler_similarity
from pydantic import BaseModel
from typing import List
from openai import OpenAI
import os

class PositiveVariants(BaseModel):
    positive_variants: List[str]

class NegativeVariants(BaseModel):
    negative_variants: List[str]

class CompanyVariants(BaseModel):
    original_name: str
    positive_variants: List[str]
    negative_variants: List[str]


def generate_company_variants(row: pd.Series) -> CompanyVariants:
    client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
    
    # First prompt for positive variants
    positive_prompt = f"""
    You are a corporate data verification assistant with knowledge of real corporate filings, stock exchange listings, and official documents (up to your training data cutoff).
    
    Task: Generate up to 5 REAL and VERIFIABLE name variations for this SAME company. Each variation must be an alternate form of the official name found in:
    - Canadian Government filings
    - Canadian Government or historical business registries
    - Official stock exchange listings (e.g., TSX, NYSE)
    - The company's own legal or historical documents
    
    Company Details:
    - Name: '{row['Name']}'
    - Industry: {row['Industry']}
    - Sector: {row['Sector']}
    - Business Type: {row['Notes']}
    
    Requirements for POSITIVE VARIANTS:
    1. All 5 variants must refer to the SAME entity, just documented differently.
    2. Variants should have a LOW string similarity (via Levenshtein or Jaro-Winkler) when compared to the original name (e.g., using abbreviations, historical names, or alternate legal forms).
    3. If you CANNOT confirm an additional variant, return fewer entries and use: "No additional verified variant found" for the remainder.
    4. Do NOT fabricate new or fictional names—only use known, verifiable aliases if they exist.
    
    Return ONLY valid JSON with a single key:
    {{
      "positive_variants": [
         "...",
         "...",
         ...
      ]
    }}
    No extra text or explanation outside the JSON.
    """

    # Second prompt for negative variants
    negative_prompt = f"""
    You are a corporate data verification assistant with knowledge of real Canadian corporate filings, Canadian stock exchange listings, and official documents (up to your training data cutoff).
    
    Task: Generate up to 5 REAL company names that are DISTINCT from '{row['Name']}' but have HIGH Levenshtein or Jaro-Winkler string similarity scores. Despite the similarity, a human can tell they are different entities.
    
    Company Details:
    - Name: '{row['Name']}'
    - Industry: {row['Industry']}
    - Sector: {row['Sector']}
    - Business Type: {row['Notes']}
    
    Requirements for NEGATIVE VARIANTS:
    1. Each must be an actual, verifiably different company (past or present), referenced in credible sources (e.g., official Canadian registries).
    2. Names should share significant overlap in words or structure with '{row['Name']}' to ensure high lexical similarity, but must refer to a separate legal entity.
    3. If you cannot confirm all 5, provide fewer and use "No additional verified variant found" for the rest.
    4. Do NOT fabricate or invent company names.
    
    Return ONLY valid JSON with a single key:
    {{
      "negative_variants": [
         "...",
         "...",
         ...
      ]
    }}
    No extra text or explanation outside the JSON.
    """

    
    pos_response = client.beta.chat.completions.parse(
        model="gpt-4o",
        response_format=PositiveVariants,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates real business names as they might appear in different corporate databases and registries."},
            {"role": "user", "content": positive_prompt}
        ]
    )
    
    neg_response = client.beta.chat.completions.parse(
        model="gpt-4o",
        response_format=NegativeVariants,
        messages=[
            {"role": "system", "content": "You are a helpful assistant that generates real business names as they might appear in different corporate databases and registries."},
            {"role": "user", "content": negative_prompt}
        ]
    )
    # Parse the JSON responses
    pos_variants = eval(pos_response.choices[0].message.content)
    neg_variants = eval(neg_response.choices[0].message.content)
    
    # Combine results
    return CompanyVariants(
        original_name=row['Name'],
        positive_variants=pos_variants['positive_variants'],
        negative_variants=neg_variants['negative_variants']
    )


In [83]:
# Get first company from our dataset
test_company = companies.iloc[0]
print("Testing with company:", test_company['Name'])

try:
    # Generate variants
    variants = generate_company_variants(test_company)
    
    # Print results
    print("\nOriginal Name:", variants.original_name)
    print("\nPositive Variants:")
    for v in variants.positive_variants:
        print(f"- {v}")
    print("\nNegative Variants:")
    for v in variants.negative_variants:
        print(f"- {v}")
        
except Exception as e:
    print(f"Error occurred: {str(e)}")
    print("Response structure:")
    print("Positive response type:", type(pos_response))
    print("Negative response type:", type(neg_response))


Testing with company: 1-800-GOT-JUNK?

Original Name: 1-800-GOT-JUNK?

Positive Variants:
- The Rubbish Boys Disposal Service Inc.
- RBDS Junk Removal Inc.
- RBDS Rubbish Removal

Negative Variants:
- 1800Junk.com Inc.
- JUSTJUNK Inc.
- Junk Ninja
- Bin There Dump That
- Green Junk Removal Inc.


In [89]:
def create_matched_dataset(companies_df: pd.DataFrame, n_samples=None) -> pd.DataFrame:
    """
    For each company:
      - Generate up to 5 positive variants and 5 negative variants.
      - Filter out "No additional verified variant found".
      - From the remaining positive variants, pick the one with the lowest Jaro-Winkler similarity.
      - From the remaining negative variants, pick the one with the highest Jaro-Winkler similarity.
      - Output two rows per company: one labeled '1' (positive) and one labeled '0' (negative).
    """
    
    if n_samples:
        companies_df = companies_df.head(n_samples)
    
    all_pairs = []
    
    for idx, row in companies_df.iterrows():
        try:
            # 1. Generate variants from GPT
            variants = generate_company_variants(row)
            original_name = variants.original_name
            
            # 2. Filter out placeholders from both lists
            pos_variants_filtered = [
                v for v in variants.positive_variants 
                if v.strip().lower() != "no additional verified variant found"
            ]
            neg_variants_filtered = [
                v for v in variants.negative_variants 
                if v.strip().lower() != "no additional verified variant found"
            ]
            
            # If there are no valid positive or negative variants, skip this company
            if not pos_variants_filtered or not neg_variants_filtered:
                print(f"Skipping {original_name}: not enough valid variants.")
                continue
            
            # 3. Compute Jaro-Winkler similarities and pick best positive & negative
            pos_sims = [
                (v, jaro_winkler_similarity(original_name.lower(), v.lower())) 
                for v in pos_variants_filtered
            ]
            # Best positive = the one with the *lowest* JW similarity
            best_positive, best_positive_sim = min(pos_sims, key=lambda x: x[1])
            
            neg_sims = [
                (v, jaro_winkler_similarity(original_name.lower(), v.lower())) 
                for v in neg_variants_filtered
            ]
            # Best negative = the one with the *highest* JW similarity
            best_negative, best_negative_sim = max(neg_sims, key=lambda x: x[1])
            
            # 4. Append results to the dataset
            all_pairs.append({
                'Name_One': original_name,
                'Name_Two': best_positive,
                'Similarity': best_positive_sim,
                'Label': 1  # same entity
            })
            all_pairs.append({
                'Name_One': original_name,
                'Name_Two': best_negative,
                'Similarity': best_negative_sim,
                'Label': 0  # different entity
            })
            
            print(f"Processed company {idx+1}/{len(companies_df)}: {original_name}")
        
        except Exception as e:
            print(f"Error processing {row['Name']}: {str(e)}")
            continue
    
    # 5. Convert to DataFrame
    df = pd.DataFrame(all_pairs)
    print(f"\nCreated dataset with {len(df)} row(s).")
    return df

# Example usage
test_df = create_matched_dataset(companies, n_samples=3)
test_df

Processed company 1/3: 1-800-GOT-JUNK?
Processed company 2/3: Norda Stelo
Processed company 3/3: 3Way International Logistics

Created dataset with 6 row(s).


,Name_One,Name_Two,Similarity,Label
0,1-800-GOT-JUNK?,RBDS Rubbish Boys Disposal Service Inc.,0.228205,1
1,1-800-GOT-JUNK?,1-800-GOT-DEBRIS,0.858333,0
2,Norda Stelo,Stelo Engineering Ltd.,0.508658,1
3,Norda Stelo,Nordic Steel Ltd.,0.869519,0
4,3Way International Logistics,Three Way International Incorporated,0.662578,1
5,3Way International Logistics,Waybest Logistics International,0.809282,0


In [92]:
##############################################
# 3. Generate final dataset for ALL ROWS     #
##############################################
all_rows = []  # will hold final dataset records

for idx, row in companies_df.iterrows():
    # Step A: Generate up to 5 positive & negative variants
    try:
        variants = generate_company_variants(row)
    except Exception as e:
        print(f"Error generating variants for index={idx}, Name='{row['Name']}': {str(e)}")
        continue
    
    # Step B: Filter out placeholders
    pos_filtered = [v for v in variants.positive_variants
                    if v.strip().lower() != "no additional verified variant found"]
    neg_filtered = [v for v in variants.negative_variants
                    if v.strip().lower() != "no additional verified variant found"]
    
    if not pos_filtered or not neg_filtered:
        print(f"Skipping index={idx}, Name='{row['Name']}' - no valid variants.")
        continue
    
    original = variants.original_name  # same as row['Name']

    # Step C: Calculate Jaro-Winkler similarities
    pos_sims = [(v, jaro_winkler_similarity(original.lower(), v.lower())) for v in pos_filtered]
    neg_sims = [(v, jaro_winkler_similarity(original.lower(), v.lower())) for v in neg_filtered]
    
    # Best positive = lowest Jaro-Winkler
    best_positive, pos_sim = min(pos_sims, key=lambda x: x[1])
    # Best negative = highest Jaro-Winkler
    best_negative, neg_sim = max(neg_sims, key=lambda x: x[1])
    
    # Step D: Append 2 new rows (label=1 for positive, label=0 for negative)
    # Include original columns and add CompanyID
    # We'll call them Name_One (original) and Name_Two (variant)
    row_base = {
        "CompanyID": idx,
        "Name": row['Name'],
        "Industry": row['Industry'],
        "Sector": row['Sector'],
        "Notes": row['Notes'],
        "Headquarters": row['Headquarters']
    }
    
    # 1) Positive row
    pos_row = {
        **row_base,
        "Name_One": original,
        "Name_Two": best_positive,
        "Label": 1
    }
    all_rows.append(pos_row)
    
    # 2) Negative row
    neg_row = {
        **row_base,
        "Name_One": original,
        "Name_Two": best_negative,
        "Label": 0
    }
    all_rows.append(neg_row)
    
    print(f"Processed idx={idx}, Name='{row['Name']}' with 2 new rows.")

# Step E: Convert final list to DataFrame
final_df = pd.DataFrame(all_rows)
print(f"\nFinal dataset size: {len(final_df)} rows.")

Processed idx=0, Name='1-800-GOT-JUNK?' with 2 new rows.
Processed idx=1, Name='Norda Stelo' with 2 new rows.
Processed idx=2, Name='3Way International Logistics' with 2 new rows.
Skipping index=3, Name='A Buck or Two' - no valid variants.
Processed idx=4, Name='A&W' with 2 new rows.
Processed idx=5, Name='ABS Friction[10]' with 2 new rows.
Processed idx=6, Name='Access Communications' with 2 new rows.
Processed idx=7, Name='Ache Records' with 2 new rows.
Processed idx=8, Name='Addition Elle' with 2 new rows.
Processed idx=9, Name='Advanced Cyclotron Systems' with 2 new rows.
Processed idx=10, Name='Affinity Credit Union' with 2 new rows.
Processed idx=11, Name='AHED' with 2 new rows.
Processed idx=12, Name='Air Canada' with 2 new rows.
Processed idx=13, Name='Air Miles' with 2 new rows.
Skipping index=14, Name='Aldo Group' - no valid variants.
Processed idx=15, Name='AldrichPears Associates' with 2 new rows.
Processed idx=16, Name='Algonquin Power & Utilities' with 2 new rows.
Process

In [93]:
final_df

,CompanyID,Name,Industry,Sector,Notes,Headquarters,Name_One,Name_Two,Label
0,0,1-800-GOT-JUNK?,Industrials,Waste & disposal services,Junk removal,Vancouver,1-800-GOT-JUNK?,RBDS Rubbish Boys Disposal Service Inc.,1
1,0,1-800-GOT-JUNK?,Industrials,Waste & disposal services,Junk removal,Vancouver,1-800-GOT-JUNK?,1-800-JUNK-PROS Ltd.,0
2,1,Norda Stelo,Industrials,Construction & Materials,Integrated projects,Quebec City,Norda Stelo,GENIVAR Inc.,1
3,1,Norda Stelo,Industrials,Construction & Materials,Integrated projects,Quebec City,Norda Stelo,Noranda Inc.,0
4,2,3Way International Logistics,Industrials,Business support services,Freight forwarding,Mississauga,3Way International Logistics,Three-Way Int'l Logistics Corp.,1
...,...,...,...,...,...,...,...,...,...
929,520,Zag Bank,Financials,Banks,Bank,Calgary,Zag Bank,B2B Bank,0
930,521,Zellers,Consumer services,Broadline retailer,Acquired by Target Canada,Brampton,Zellers,Hudson's Bay Trading Company (Zellers),1
931,521,Zellers,Consumer services,Broadline retailer,Acquired by Target Canada,Brampton,Zellers,Zellweger Analytics Canada Ltd,0
932,522,ZENN Motor Company,Consumer goods,Automobiles,Electric vehicles,Montreal,ZENN Motor Company,"Feel Good Cars, Inc.",1


In [94]:
##################################
# 4. Save to CSV (no GPT traces) #
##################################
output_file = "canadian_company_variants.csv"
final_df.to_csv(output_file, index=False)

print(f"Saved final dataset to '{output_file}'.")

Saved final dataset to 'canadian_company_variants.csv'.
